In [1]:
import tensorflow as tf

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
# 交叉熵
#cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y,1e-10,1.0))
#                               + (1-y_)*tf.log(tf.clip_by_value(1-y,1e-10,1.0)))
# tf.clip_by_value()函数可以将一个张量中的数值限制在一个范围内，利用这个函数可以保证在进行log运算时，
# 不会出现log0这样的错误，或者概率大于1的错误
v = tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0]])
with tf.Session() as sess:
    print(sess.run(tf.clip_by_value(v,2.5,4.5)))
# 将小于2.5的替换成2.5，将大于4.5的替换成4.5

[[2.5 2.5 3. ]
 [4.  4.5 4.5]]


In [10]:
# 相乘与内积
v1 = tf.constant([[1.0,2.0],[3.0,4.0]])
v2 = tf.constant([[5.0,6.0],[7.0,8.0]])
print(v1*v2) # 相乘  mul
print(tf.matmul(v1,v2)) # 矩阵乘法 内积 matmul
with tf.Session() as sess:
    print(sess.run(v1*v2))
    print(sess.run(tf.matmul(v1,v2)))

Tensor("mul_4:0", shape=(2, 2), dtype=float32)
Tensor("MatMul_3:0", shape=(2, 2), dtype=float32)
[[ 5. 12.]
 [21. 32.]]
[[19. 22.]
 [43. 50.]]


In [11]:
# 取均值，因上面得到的交叉熵是1个batch的，需要平均下，取这个batch的平均交叉熵作为结果
v3 = tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0]])
with tf.Session() as sess:
    print(sess.run(tf.reduce_mean(v3)))

3.5


In [ ]:
# tensorflow有实现cross_entropy
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=,logits=)

In [16]:
# 自定义损失函数
v1 = tf.constant([1.0,2.0,3.0,4.0])
v2 = tf.constant([4.0,3.0,2.0,1.0])

sess = tf.InteractiveSession()
print(tf.greater(v1,v2).eval())
# [False False  True  True]
print(tf.where(tf.greater(v1,v2),v1,v2).eval())

sess.close()

[False False  True  True]
[4. 3. 3. 4.]


D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [18]:
from numpy.random import RandomState
batch_size = 8

x = tf.placeholder(tf.float32,[None,2],name='x-input')

y_ = tf.placeholder(tf.float32,[None,1],name='y-input')

w1 = tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))
y = tf.matmul(x,w1)

# 定义预测多了和少了的成本
loss_less = 10
loss_more = 1

loss = tf.reduce_mean(tf.where(tf.greater(y,y_),(y-y_)*loss_more,(y_-y)*loss_less))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size,2)
Y = [[x1+x2+rdm.rand()/10.0-0.5] for x1,x2 in X]


array([[4.17022005e-01, 7.20324493e-01],
       [1.14374817e-04, 3.02332573e-01],
       [1.46755891e-01, 9.23385948e-02],
       [1.86260211e-01, 3.45560727e-01],
       [3.96767474e-01, 5.38816734e-01],
       [4.19194514e-01, 6.85219500e-01],
       [2.04452250e-01, 8.78117436e-01],
       [2.73875932e-02, 6.70467510e-01],
       [4.17304802e-01, 5.58689828e-01],
       [1.40386939e-01, 1.98101489e-01],
       [8.00744569e-01, 9.68261576e-01],
       [3.13424178e-01, 6.92322616e-01],
       [8.76389152e-01, 8.94606664e-01],
       [8.50442114e-02, 3.90547832e-02],
       [1.69830420e-01, 8.78142503e-01],
       [9.83468338e-02, 4.21107625e-01],
       [9.57889530e-01, 5.33165285e-01],
       [6.91877114e-01, 3.15515631e-01],
       [6.86500928e-01, 8.34625672e-01],
       [1.82882773e-02, 7.50144315e-01],
       [9.88861089e-01, 7.48165654e-01],
       [2.80443992e-01, 7.89279328e-01],
       [1.03226007e-01, 4.47893526e-01],
       [9.08595503e-01, 2.93614148e-01],
       [2.877753

In [20]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for i in range(5001):
        start = (i*batch_size) % dataset_size
        end = min(start + batch_size,dataset_size)
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})

    print(sess.run(w1))

[[0.6457671 ]
 [0.82601607]]


In [ ]:
# 指数衰减学习率
global_step = tf.Variable(0)
# 通过exponential_decay函数生成学习率
learning_rate = tf.train.exponential_decay(0.1,global_step=global_step,100,0.96,staircase=True)
# 使用指数衰减的学习率。在minimize函数中传入global_step将自动更新global_step参数，从而使得学习率也相应更新
learning_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

# 设定初始学习率为0.1，staircase=True设置为True，每迭代10次，学习率乘以0.96.